In [ ]:
import json
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
DAY = "day_3" # 14.07

In [ ]:
def get_dataframe(f_name, save_xlsx=False):
    with open(f"raw_data/{f_name}.bmerawdata") as f:
        j = json.load(f)
    column_names = [el["name"] for el in j["rawDataBody"]["dataColumns"]]
    df = pd.DataFrame(j["rawDataBody"]["dataBlock"], columns=column_names)

    if save_xlsx:
        df.to_excel(f"raw_data/{f_name}.xlsx")

    return df

In [ ]:
def build_sensor_data(df):
    df["Date"] = pd.to_datetime(
        df["Real time clock"],
        unit="s",
        utc=True).map(lambda x: x.tz_convert("Europe/Istanbul"))
    df["Date"] = df["Date"].dt.tz_localize(None)

    sensor_indexes = sorted(df["Sensor Index"].unique())

    sensors = {}
    for i in sensor_indexes:
        sensors[i] = {}
        sensor = df[df["Sensor Index"] == i]
        heater_indexes = sorted(sensor["Heater Profile Step Index"].unique())
        for j in heater_indexes:
            sensors[i][j] = sensor[sensor["Heater Profile Step Index"] == j]

    return sensors

In [ ]:
def build_raw_data_pickle(save_pickle=None):
    sensor_data = {}
    for matrix in ["air", "sample"]:
        df = get_dataframe(f"{DAY}/{matrix}")
        sensor_data[f"{matrix}"] = build_sensor_data(df)

    if save_pickle is not None:
        with open(f"{save_pickle}.pkl", "wb") as f:
            pickle.dump(sensor_data, f)

    return sensor_data

In [ ]:
sensor_data = build_raw_data_pickle(save_pickle=f"data/{DAY}/raw")

In [ ]:
sensor_data["air"][0][0]

In [ ]:
def plot_sensor(matrix, sensor_idx, sensor_data):
    plt.figure(figsize=(8, 4))
    plt.title(f"Matrix {matrix}, Sensor {sensor_idx}")
    plt.yscale("log")
    for hp_idx, val in sensor_data[f"{matrix}"][sensor_idx].items():
        resistance = val["Resistance Gassensor"].values
        time_ms = val["Time Since PowerOn"].values
        plt.plot(time_ms, resistance, label=f"HP {hp_idx}")
    plt.legend()
    plt.show()

In [ ]:
plot_sensor("air", 0, sensor_data)

In [ ]:
plot_sensor("sample", 0, sensor_data)